In [1]:
import numpy as np
from qiskit import QuantumCircuit

from helper import *
from qite import aer_sim as backend
from qite import log_qite_run

import time

# Real Hamiltonian in Z basis
For this case, we only need to consider Hamiltonian terms that contain odd number of Ys.

This changes the alist data structure so that it only stores the amplitudes for said Odd Y Pauli strings. for `nbits` qubits, a list of the odd Y pauli strings can be found with `helper.odd_y_pauli_strings(nbits)`.

In [6]:
def y_propogate(qc, alist):
    for t in range(len(alist)):
        active = alist[t][1]
        nbits = len(active)
        odd_y_paulis = odd_y_pauli_strings(nbits)
        for i in range(len(odd_y_paulis)):
            angle = np.real(alist[t][0][i])
            if np.abs(angle) > 1e-5:
                pauli_string_exp(qc, active, odd_y_paulis[i], angle)
                qc.barrier()

In [7]:
nbits = 2
odd_ys = odd_y_pauli_strings(nbits)
print(odd_ys)
print(len(odd_ys))

[2, 6, 8, 9, 11, 14]
6


In [9]:
alist = [ [ [np.pi, np.pi/2, np.pi/4, np.pi/8, np.pi/16, np.pi/32], [0,1] ] ]
qc = QuantumCircuit(nbits, nbits)
y_propogate(qc, alist)
qc.draw()

┌───────┐ ░ ┌─────────┐                     ┌──────────┐ ░             ░ »
q_0: ┤ Ry(π) ├─░─┤ Rx(π/2) ├──■───────────────■──┤ Rx(-π/2) ├─░─────────────░─»
     └───────┘ ░ └──┬───┬──┘┌─┴─┐┌─────────┐┌─┴─┐└──┬───┬───┘ ░ ┌─────────┐ ░ »
q_1: ──────────░────┤ H ├───┤ X ├┤ Rz(π/2) ├┤ X ├───┤ H ├─────░─┤ Ry(π/4) ├─░─»
               ░    └───┘   └───┘└─────────┘└───┘   └───┘     ░ └─────────┘ ░ »
c: 2/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌───┐                           ┌───┐     ░                 »
«q_0: ───┤ H ├─────■───────────────■─────┤ H ├─────░──────────────■──»
«     ┌──┴───┴──┐┌─┴─┐┌─────────┐┌─┴─┐┌──┴───┴───┐ ░ ┌─────────┐┌─┴─┐»
«q_1: ┤ Rx(π/2) ├┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rx(-π/2) ├─░─┤ Rx(π/2) ├┤ X ├»
«     └─────────┘└───┘└─────────┘└───┘└──────────┘ ░ └─────────┘└───┘»
«c: 2/═══════════════════════════════════════════════════════════════»
«                                                                    »
«                                   ░ ┌─────────┐                      »
«q_0: ──────────────■───────────────░─┤ Rx(π/2) ├──■────────────────■──»
«     ┌──────────┐┌─┴─┐┌──────────┐ ░ └─────────┘┌─┴─┐┌──────────┐┌─┴─┐»
«q_1: ┤ Rz(π/16) ├┤ X ├┤ Rx(-π/2) ├─░────────────┤ X ├┤ Rz(π/32) ├┤ X ├»
«     └──────────┘└───┘└──────────┘ ░            └───┘└──────────┘└───┘»
«c: 2/═════════════════════════════════════════════════════════════════»
«                                                                      »
«     ┌──────────┐ ░ 
«q_0: ┤ Rx(-π/2) ├─░─
«     └──────────┘ ░ 
«q_1: ─────────────░─
«                  ░ 
«c: 2/═══════════════
«

In [19]:
def y_tomography(hm, alist, qbits, nbits, backend, shots=1024):
    nactive = len(qbits)
    odd_ys = odd_y_pauli_strings(nactive)
    nops = len(odd_ys)
    
    sigma_expectation = {}
    
    # figure out which expectations are required
    keys = []
    for p1 in odd_ys:
        for p2 in odd_ys:
            keys.append((p1,p2))
        for p2 in hm[0]:
            if not ((p1,p2) in keys):
                keys.append((p1,p2))
    

y_tomography([[7],[0.5],[0,1]], None, [0,1], 2, None, None)

[(2, 2),
 (2, 6),
 (2, 8),
 (2, 9),
 (2, 11),
 (2, 14),
 (2, 7),
 (6, 2),
 (6, 6),
 (6, 8),
 (6, 9),
 (6, 11),
 (6, 14),
 (6, 7),
 (8, 2),
 (8, 6),
 (8, 8),
 (8, 9),
 (8, 11),
 (8, 14),
 (8, 7),
 (9, 2),
 (9, 6),
 (9, 8),
 (9, 9),
 (9, 11),
 (9, 14),
 (9, 7),
 (11, 2),
 (11, 6),
 (11, 8),
 (11, 9),
 (11, 11),
 (11, 14),
 (11, 7),
 (14, 2),
 (14, 6),
 (14, 8),
 (14, 9),
 (14, 11),
 (14, 14),
 (14, 7)]